<a href="https://colab.research.google.com/github/vlykamol/MTP/blob/main/m_machines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import math
import numpy as np
PI = math.pi
from scipy.integrate import quad
from scipy.linalg import expm
import random

In [3]:
def AI(Kd, J, Kt, Fd, GrI, Ke, L, R, Ts):
  return np.array([[-Kd/J, Kt/J, -(Fd * GrI)/J], [-Ke/L, -R/L, 0], [(Ts * GrI)/(2 * math.pi), 0, 0]])

def BI(GrI, J, L):
  return np.array([[0, -GrI/J], [1/L, 0], [0, 0]])

def SI(AI, h):
  try:
    AI_inv = np.linalg.inv(AI) # calulate inverse of AI
    exp_AI_h = expm(AI * h) # exponential of AI * h
    I = np.eye(len(AI)) # Identity matrix of size AI
    Si = N @ AI_inv @ (exp_AI_h - I)
    return Si
  except np.linalg.LinAlgError:
    print("Error: The matrix Ai is singular and cannot be inverted.")
    return None

N = np.array([[0, 1, 0], [0, 0, 0]])

def RI(AI, BI, h):
  try:
    AI_inv = np.linalg.inv(AI) # calulate inverse of AI
    exp_AI_h = expm(AI * h) # exponential of AI * h
    I = np.eye(len(AI)) # Identity matrix of size AI

    mTerm = (AI_inv @ exp_AI_h) - AI_inv - I * h

    RI = N @ AI_inv @ mTerm @ BI
    return RI
  except np.linalg.LinAlgError:
    print("Error: The matrix Ai is singular and cannot be inverted.")
    return None

def Ei(x, u, SI, RI):
  fTerm = np.transpose(u) @ SI @ x
  sTerm = np.transpose(u) @ RI @ u
  return fTerm + sTerm

In [4]:
def Fi(Ai, Bi, x, u, h):
  def integrand(s): # Modified integrand to take index i
    return expm(Ai * s) @ Bi

  Phi_i = expm(Ai * h)
  # Calculate Gamma_i for each element of Bi
  Gamma_i = np.zeros_like(Bi)
  for i in range(Bi.shape[0]):
    for j in range(Bi.shape[1]):
      Gamma_i[i, j], _ = quad(lambda s: integrand(s)[i, j], 0, h)

  # print("phi_i", Phi_i)
  # print("Phi_i @ x", Phi_i @ x)
  # print("Gamma_i", Gamma_i)
  # print("Gamma_i @ u", Gamma_i @ u)
  return Phi_i @ x + Gamma_i @ u

In [ ]:
h = 0.2 # time step
tD = 1 # target depth
Va = 10 # input voltage

In [ ]:
J = 0.1
Kd = 0.25
Ke = 1
Kt = 1
Ts = 1
L = 2.5 * 10e-4
R = 1
Fd = 0.1
Fs = 0.2
Va = 20
G = [1, 2]

In [ ]:
# process constants
A = [AI(Fd=Fd, GrI=x, J=J, Kd=Kd, Ke=Ke, Kt=Kt, L=L, R=R, Ts=Ts) for x in G]
B = [BI(GrI=x, J=J, L=L) for x in G]

In [ ]:
A, B

In [ ]:
x = np.array([0, 0, 0]) # initial state
u = np.array([Va, Fs]) # initial control input

In [ ]:
x_k1 = Fi(Ai = A[0], Bi=B[0], x=x, u=u, h=h)
print(x_k1, np.shape(x_k1))

In [ ]:
x_k1 = [x_k1[0], x_k1[1], 0.25]
x_k2 = Fi(Ai = A[1], Bi=B[1], x=x_k1, u=u, h=h)
print(x_k2, np.shape(x_k2))

In [ ]:
x_k1 = Fi(Ai = A[1], Bi=B[1], x=x, u=u, h=h) # time step = 1 m = 1, n = 3(mode = 2)
print(x_k1, np.shape(x_k1))
x_k1 = [x_k1[0], x_k1[1], 0.35]

x_k2 = Fi(Ai = A[0], Bi=B[0], x=x_k1, u=u, h=h) # time step = 2 m = 1, n = 2(mode = 1)
print(x_k2, np.shape(x_k2))
temp = math.floor(x_k2[2] * 1000/250)
temp = temp * 250 / 1000
x_k2 = [x_k2[0], x_k2[1], temp]
print(x_k2)

x_k3 = Fi(Ai = A[1], Bi=B[1], x=x_k2, u=u, h=h) # time step = 3 m = 1, n = 2(mode = 1)
print(x_k3, np.shape(x_k3))
temp = math.floor(x_k3[2] * 1000/350)
print(temp)
temp = temp * 350 / 1000
print(temp)

In [ ]:
x_k2 = Fi(Ai = A[1], Bi=B[1], x=x_k1, u=u, h=h)
print(x_k2, np.shape(x_k2))

In [ ]:
x_k1 = Fi(Ai = A[1], Bi=B[1], x=x, u=u, h=h)
print(x_k1, np.shape(x_k1))
x_k2 = Fi(Ai = A[1], Bi=B[1], x=x_k1, u=u, h=h)
print(x_k2, np.shape(x_k2))

# one machine scheduling


In [ ]:
# prompt: create a node structure which holds information like one list, one state (in list formate) and one cost

class Node:
  def __init__(self, state, path, cost):
    self.state = state  # List representing the state
    self.path = path    # List of actions taken to reach this state
    self.cost = cost    # Cost to reach this state

  def print_node(self):
    print("State:", self.state)
    print("Path:", self.path)
    print("Cost:", self.cost)

In [ ]:
# machine m ranges from (0,1,2..n-1) having gear ratios
def applyMachine(m, node):
  x = node.state
  I = node.path
  e = node.cost

  #state update
  Ai = A[m]
  Bi = B[m]

  x_1 = Fi(Ai, Bi, x, u, h)

  #path update
  I.append(m)

  #cost update
  Si = SI(Ai, h)
  Ri = RI(Ai, Bi, h)
  ei = Ei(x, u, Si, Ri)
  e += ei

  print(x_1)
  print(I)
  print(e)
  return Node(x_1, I, e)

In [ ]:
m = 3 # no. of machines
n = 2 # no. of modes

In [ ]:
sol = []
pSol = [applyMachine(i, Node(x, [], 0)) for i in range(0, len(G))]

In [ ]:
D_k = [] # gradient depth factors

In [ ]:
class sol_list:
  def __init__(self, m):
    self.sol = {

    }

In [ ]:
while pSol: # while loop
  _pSol = []  # empty list
  for i in pSol: # for all partial solution
    _pSol += [applyMachine(j, i) for j in range(0, len(G))] # apply all machine for one time step

  # remove partial solution that are dominated by those already in sol
  for i in _pSol:
    for j in sol:
      if i.cost >= j.cost:
        _pSol.remove(i)
        break

  # move complete solution to sol
  pSol = []
  print("len of _psol", len(_pSol), "len of pSol", len(pSol))
  for i in _pSol:
    x = i.state
    if x[2] >= tD:
      sol.append(i)
    else:
      pSol.append(i) # Replace pSol with remaining from _pSol

#New implementation

In [5]:
class SL:
    def __init__(self, mode, power):
        self.mode = mode  # Mode associated with the service level
        self.power = power  # Power associated with the mode

    def __repr__(self):
        """Representation of a service level."""
        return f"(M: {self.mode}, P: {self.power})"

In [6]:
class Machine:
    def __init__(self, name):
        self.name = name  # (name or ID)
        self.service_levels = []  # service levels (SLs)

    def add_sl(self, mode, power):
        """Add a ServiceLevel to the machine."""
        sl = SL(mode, power)
        self.service_levels.append(sl)

    def get_sl(self):
        """Return all the service levels."""
        return self.service_levels

    def __repr__(self):
        """Representation of the machine."""
        return f"M(ID: {self.name}, SLs: {self.service_levels})"

In [7]:
class Node:
    def __init__(self, power, mode, work_done):
        self.power = power  # Total power
        self.work_done = work_done  # Work done
        self.modes = []  # List to store tuples of (mode, power)
        self.next = None  # Pointer to the next node

    def add_sl(self, mode, power):
        """Add a tuple of (mode, power) to the list."""
        sl = SL(mode, power)
        self.modes.append(sl)

    def get_modes(self):
        """Return all the modes and associated powers."""
        return self.modes

    def __repr__(self):
        """Represent the node details."""
        return f"Node(p: {self.power}, w: {self.work_done}, Modes: {self.modes})"


In [22]:
class LinkedList:
    def __init__(self):
        self.head = None  # Head of the list (initially empty)
        self.tail = None
        self.len = 0;

    # Method to add a new node at the end
    def append(self, Node):
        self.len += 1
        # If the list is empty, make the new node the head
        if self.head is None:
            self.head = Node
            self.tail = Node
            return
        else:
            self.tail.next = Node
            self.tail = Node

    # Method to get size of linked list
    def __len__(self):
        return self.len

    def copy(self):
        new_list = LinkedList()
        curr = self.head
        while curr:
            new_node = Node(power=curr.power, mode=None, work_done=curr.work_done)
            new_node.modes = curr.modes.copy()
            new_list.append(new_node)
            curr = curr.next
        return new_list

    # Method to print the list
    def __repr__(self):
        curr = self.head
        nodes = [] # create a list to hold the string representation of the nodes
        while curr:
            nodes.append(str(curr)) # add string representation of node to list
            curr = curr.next
        return " -> ".join(nodes) + " -> None" # return the formatted string


In [56]:
def create_machine(id, gMin, gRange, num_gears):
    """
      args:
        id: machine id,
        gMin: min value of gear ratio,
        gRange: range of gear ratio,
        num_gears: number of gears
    """
    machine = Machine(name=f"{id}")
    previous_power = 0
    gears = [gMin + x * gRange for x in range(num_gears)]
    for j in gears:
        power = random.randint(max(30, previous_power + 1), previous_power + 30)
        machine.add_sl(j, power)
        previous_power = power
    return machine

In [81]:
# Script to create 3 machines and print their details
machines = [create_machine(x, 1, 3, 3) for x in range(1, 4)]

# Print details of each machine
for machine in machines:
    print(machine)

M(ID: 1, SLs: [(M: 1, P: 30), (M: 4, P: 55), (M: 7, P: 59)])
M(ID: 2, SLs: [(M: 1, P: 30), (M: 4, P: 51), (M: 7, P: 56)])
M(ID: 3, SLs: [(M: 1, P: 30), (M: 4, P: 51), (M: 7, P: 56)])


In [79]:
T = 1; # total time
P = 150; # total power at time t
h = 0.2 # time step
tD = 1 # target depth
Va = 10 # input voltage

In [13]:
J = 0.1
Kd = 0.25
Ke = 1
Kt = 1
Ts = 1
L = 2.5 * 10e-4
R = 1
Fd = 0.1
Fs = 0.2

In [15]:
# machine m ranges from (0,1,2..n-1) having gear ratios
def applyMode(m, state, input):
  #state update
  Ai = AI(Fd=Fd, GrI=m, J=J, Kd=Kd, Ke=Ke, Kt=Kt, L=L, R=R, Ts=Ts)
  Bi = BI(GrI=m, J=J, L=L)
  x_1 = Fi(Ai, Bi, state, input, h)
  return x_1

In [28]:
def getMid(llist):
        if llist.head is None:
          return None
        slow = llist.head
        fast = llist.head
        while fast and fast.next:
            slow = slow.next
            fast = fast.next.next
        return slow

def merge(left, right):
  result = LinkedList()
  left_node = left.head
  right_node = right.head

  while left_node and right_node:
      if left_node.work_done <= right_node.work_done:
          result.append(left_node)
          left_node = left_node.next
      else:
          result.append(right_node)
          right_node = right_node.next

  # Append remaining nodes, if any
  while left_node:
      result.append(left_node)
      left_node = left_node.next
  while right_node:
      result.append(right_node)
      right_node = right_node.next

  return result

def sort(llist):
    if llist.head is None or llist.head.next is None:
        return llist

    # Get the middle of the list and split it
    mid = getMid(llist)

    # Split the linked list into two halves
    left_half = LinkedList()
    left_half.head = llist.head
    current = left_half.head
    while current.next != mid:
        current = current.next
    current.next = None  # Break the first half

    right_half = LinkedList()
    right_half.head = mid

    # Recursively sort the two halves
    left_sorted = sort(left_half)
    right_sorted = sort(right_half)

    # Merge the two sorted halves
    return merge(left_sorted, right_sorted)

In [54]:
def pruneNodes(llist):
  head = llist.head
  if not head:
        return None
  llist = sort(llist)
  return llist

In [85]:
x = [0, 0, 0]
u = [Va, Fs]
states = [x for j in range(0, len(machines))]
inputs = [u for j in range(0, len(machines))]

for t in range(0, int(T / h) + 1):
  current_time = t * h
  print("time : ", current_time)

  lam = Node(power=0, mode=None, work_done=0)
  psi = LinkedList()
  psi.append(lam)

  for i, m in enumerate(machines):
      print("machine : ", m.name)
      psi_prev = psi.copy()
      g = len(psi_prev)
      lls = []
      state = states[i]
      input = inputs[i]
      for k in m.service_levels:
        phi_k = LinkedList()
        curr = psi_prev.head
        while curr:
          p = curr.power + k.power
          if p > P :
            curr = curr.next
            continue
          _state = applyMode(k.mode, state, input)
          modes = curr.get_modes().copy()
          wd = curr.work_done + _state[2]
          # create a new node
          lam = Node(power=p, mode=0, work_done=wd)
          lam.modes = modes
          lam.add_sl(k.mode, k.power)
          phi_k.append(lam) # append to kth linkedlist
          curr = curr.next

        lls.append(phi_k) # add kth linkedlist to merge

      _psi = LinkedList()
      for p in lls:
        curr = p.head
        while curr:
          _psi.append(curr)
          curr = curr.next

      # pruning nodes
      psi = _psi
      _psi = pruneNodes(_psi)
      print("psi :: ", _psi)
      tNode = _psi.tail
  # updating final state
  for i, mode in enumerate(tNode.modes):
      states[i] = applyMode(mode.mode, states[i], inputs[i])
  print("states :: ", states)
  print("end of one time step")


time :  0.0
machine :  1
psi ::  Node(p: 30, w: 0.15670743458895728, Modes: [(M: 1, P: 30)]) -> Node(p: 55, w: 0.5839428159767044, Modes: [(M: 4, P: 55)]) -> Node(p: 59, w: 0.9404543955942447, Modes: [(M: 7, P: 59)]) -> None
machine :  2
psi ::  Node(p: 60, w: 0.31341486917791456, Modes: [(M: 1, P: 30), (M: 1, P: 30)]) -> Node(p: 85, w: 0.7406502505656617, Modes: [(M: 4, P: 55), (M: 1, P: 30)]) -> Node(p: 81, w: 0.7406502505656617, Modes: [(M: 1, P: 30), (M: 4, P: 51)]) -> Node(p: 89, w: 1.097161830183202, Modes: [(M: 7, P: 59), (M: 1, P: 30)]) -> Node(p: 86, w: 1.097161830183202, Modes: [(M: 1, P: 30), (M: 7, P: 56)]) -> Node(p: 106, w: 1.1678856319534088, Modes: [(M: 4, P: 55), (M: 4, P: 51)]) -> Node(p: 110, w: 1.5243972115709492, Modes: [(M: 7, P: 59), (M: 4, P: 51)]) -> Node(p: 111, w: 1.5243972115709492, Modes: [(M: 4, P: 55), (M: 7, P: 56)]) -> Node(p: 115, w: 1.8809087911884894, Modes: [(M: 7, P: 59), (M: 7, P: 56)]) -> None
machine :  3
psi ::  Node(p: 90, w: 0.470122303766871

In [ ]:

#   p = P # total power availbale at time t